In [1]:
from Designer import DesignNotes, SST, Part, show, gvars, se_split
import math
sind = lambda deg: math.sin(math.radians(deg))  # return sin of angle expressed in degrees
cosd = lambda deg: math.cos(math.radians(deg))  # return cos of angle expressed in degrees

In [2]:
import pint                  # setup to use the module for computing with units
ureg = pint.UnitRegistry()
mm = ureg['mm']              # define symbols for the units used below
inch = ureg['inch']
kN = ureg['kN']
MPa = ureg['MPa']
ureg.default_format = '~P'

In [3]:
class Bolts(Part):
    'Bolts'   # bolt group is the same on the tongue plate and on the gusset plate.
    grade = 'ASTM A325'
    size = '3/4"'
    Fu = 825*MPa
    d = (3/4*inch).to(mm)
    hole_type = 'punched'
    hd = 22*mm            # hole diameter
    ha = hd + 2*mm        # hole allowance
    threads_intercepted = True
    nlines = 2            # a line is perpendicular to load
    nperline = 3          # number of bolts in each line
    g = 75*mm             # gauge (perpendicular to load)
    s = 75*mm             # spacing (parallel to load)

class Welds(Part):
    'Welds'
    grade = 'E49xx'
    Xu = 490*MPa
    matching = True

class Plates(Part):
    'Plates'
    grade = 'CSA G40.21 350W'
    Fy = 350*MPa
    Fu = 450*MPa

class HSS(Part):
    'HSS Column'
    grade = 'CSA G40.21 350W'
    Fy = 350*MPa
    Fu = 450*MPa
    size = 'HS127x127x13'
    D,T,A = SST.section(size,'D,T,A')
    D = D*mm
    T = T*mm
    A = A*mm*mm    

class CoverPlate(Plates):
    'Cover Plate'
    T = 10*mm
    W = 60*mm
    Lw = 90*mm      # length of weld from net section to end of HSS
    D = 6*mm        # size of weld from on HSS.
   
class TonguePlate(Plates):
    'Tongue Plate'
    T = 20*mm
    W = 280*mm
    L = 260*mm
    e = 40*mm
    D = 8*mm       # weld size
    c = 45*mm      # dist end of weld to 1st bolt line  
    Lw = L - (c + (Bolts.nlines-1)*Bolts.s + e)
    Dh = SST.section(HSS.size,'D')*mm
    ns = 10

class GussetPlate(Plates): 
    'Gusset Plate'
    T,W = TonguePlate.values('T,W')
    W2 = 110*mm
    e = 40*mm    # end distance
    D = 8*mm     # plate to column weld size
    theta = 45. 

In [4]:
size = 'HS127x127x13'
s = SST.section(size)
s

In [16]:
class CM2(object):
    
    """Context Manager 2: Injects variables into Global Name Space on enter,
    retorses them on exit."""
    
    def __init__(self, rvar, label, *itemlists, show=False, notes=None, record=False):
        self.rvar = rvar
        self.label = label
        self.itemlist = []
        for il in itemlists:
            self.itemlist.extend(il)
        self.itemlist.append((rvar,None))
        self.show = show
        self.notes = notes
        self.record = record
        if record:
            if not notes:
                raise ValueError("notes must be specified when record is True")
        self.changed_vars = {}
        self.added_vars = []
        self.gns = get_ipython().user_ns
    
    def __enter__(self):
        for k,v in self.itemlist:
            if k in self.changed_vars or k in self.added_vars:
                raise KeyError("Variable '{}' is used more than once.".format(k))
            if k in self.gns:
                self.changed_vars[k] = self.gns[k]
            else:
                self.added_vars.append(k)
            self.gns[k] = v
        
    def __exit__(self,*args):
        
        if self.show:
            values = {}
            for k,v in self.itemlist:
                values[k] = self.gns[k]
            if values:
                show(','.join([k for k,v in values.items()]),data=values,minwidth=5)
                
        if self.record:
            var = self.rvar
            val = self.gns[var]
            if self.notes.units:
                val = val.to(self.notes.units)
            self.notes.record(val,self.label)
            
        for k in self.added_vars:     # delete all added variables from GNS
            if k in self.gns:
                del self.gns[k]
        self.added_vars = []
        for k,v in self.changed_vars.items():   # restore values of other variables in GNS
            self.gns[k] = v
        self.changed_vars = {}


In [22]:
dn = DesignNotes('Vr',trace=True)
def CALC(rvar,label,*ilist,**kwds):
    self = dn
    kargs = dict(show=self.trace,record=True)
    kargs.update(kwds)
    return CM2(rvar,label,*ilist,notes=self,**kargs)
VARS = gvars

In [24]:
phiw = 0.67
with CALC('Vr','Gusset to HSS Weld',
           VARS('W,W2,theta,D',GussetPlate),
           VARS('Xu',Welds),
           VARS('L,L1,Mw,Aw')):
    L1 = W2+W*cosd(theta)
    L = (L1/sind(theta))*cosd(theta) + W*sind(theta)

    Mw = 1.0                  
    Aw = 2*L*.707*D
    Vr = 0.67*phiw*Aw*Xu*(1+sind(theta)**1.5)*Mw     # S16-14: 13.13.2.2

W     = 280       mm
W2    = 110       mm
theta = 45        
D     = 8         mm
Xu    = 490       MPa
L     = 506       mm
L1    = 308       mm
Mw    = 1         
Aw    = 5724      mm²
Vr    = 2.008e+06 MPa·mm²



    Gusset to HSS Weld: Vr = 2008000 MPa·mm²